## Quick and dirty VEV notebook

by Brian

In [17]:
from typing import Dict
from random import randrange
from tqdm import tqdm
import numpy as np
import random

### Notebook params

In [18]:
PROP_COST_ETH = 10
NUM_VOTES_BOUGHT = 2
TOTAL_VOTE_COST_ETH = 5
TOTAL_SUPPLY = 10

---

**Vote data has the following schema:**

```
{
    vote_weight: int,
    p_vote: [pFor, pAgainst, pAbstain]: float[] (must sum to 1 as it's a probability distribution),
    turnout_prob: float (how likly this voter is to actually vote)
}
```

### Toy example

In [24]:

# state of the world if you did not buy votes
vote_state_counterfactual = [
    {
        "vote_weight": 5,
        "p_vote": [.5, .3, .2],
        "turnout_prob": .8
    },
    {
        "vote_weight": 4,
        "p_vote": [.1, .8, .1],
        "turnout_prob": .8
    },
    {
        "vote_weight": 2,
        "p_vote": [.3, .3, .4],
        "turnout_prob": .01
    }
]

# state of the world if you do buy votes
# NOTE: difference here is (by assumption for the sake of the model) that you take a 0 probability of vote 
# voter and make them now a 1 with a fixed (always vote yes) vote distibuion
vote_state_vev = [
       {
        "vote_weight": 5,
        "p_vote": [.5, .3, .2],
        "turnout_prob": .8
    },
    {
        "vote_weight": 4,
        "p_vote": [.1, .8, .1],
        "turnout_prob": .8
    },
    # YOU BOUGHT THESE TWO VOTES
    {
        "vote_weight": 2,
        "p_vote": [1, 0, 0],
        "turnout_prob": 1
    }
]

In [25]:
DQ_PARAMS = {
    "minQuorumVotesBPS": 1000,
    "maxQuorumVotesBPS": 1500,
    "quorumCoefficient": 1000000
}
def bps2Uint(quorum_bps: int, total_supply: int) -> int:
    return (total_supply * quorum_bps) / 10000

def vote_win_function(total_supply: int, against_votes: int, DQ_PARAMS: Dict[str, int] ) -> int:
    againstVotesBPS = (10000 * against_votes) / total_supply;
    quorumAdjustmentBPS = (DQ_PARAMS['quorumCoefficient'] * againstVotesBPS) / 1e6;
    adjustedQuorumBPS = DQ_PARAMS["minQuorumVotesBPS"] + quorumAdjustmentBPS
    quorumBPS = min(DQ_PARAMS["maxQuorumVotesBPS"], adjustedQuorumBPS)
    return bps2Uint(quorumBPS, total_supply)

In [26]:
def simulate_vote_outcome(
    total_supply,
    voter_info
):
    N_SAMP = 1000
    outcomes = []


    for _ in tqdm(range(N_SAMP)):
        for_votes = 0
        against_votes = 0
        abstain_votes = 0
        for voter in voter_info:
            # do they vote at all?
            if random.random() > voter['turnout_prob']:
                continue

            vote = np.random.choice([0,1,2], p=voter['p_vote'])
            if vote == 0:
                for_votes += voter['vote_weight']
            if vote == 1:
                against_votes += voter['vote_weight']
            if vote == 2:
                abstain_votes += voter['vote_weight']

        treshold = vote_win_function(total_supply, against_votes, DQ_PARAMS)
        outcomes.append(for_votes > treshold)
    return np.mean(outcomes)
    

In [27]:
def simulate_vev_outcome(
    vote_state_counterfactual,
    vote_state_vev,
    prop_value_eth,
    vote_cost_eth
):
    couterfactual_outcome = simulate_vote_outcome(TOTAL_SUPPLY, vote_state_counterfactual)
    vev_outcome = simulate_vote_outcome(TOTAL_SUPPLY, vote_state_counterfactual)
    return {
        "win_rate_counterfactual": couterfactual_outcome, 
        "win_rate_vev": vev_outcome,
        "expected_profit": prop_value_eth*(vev_outcome - couterfactual_outcome) - vote_cost_eth
    }

In [28]:
simulate_vev_outcome(
    vote_state_counterfactual,
    vote_state_vev,
    PROP_COST_ETH,
    TOTAL_VOTE_COST_ETH
)

100%|██████████| 1000/1000 [00:00<00:00, 14052.59it/s]


{'win_rate_counterfactual': 0.439,
 'win_rate_vev': 0.449,
 'expected_profit': -4.9}